In [1]:
import pandas as pd 
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import numpy as np

## 1. Read in dataset

In [2]:
# This data represents a weekly retail scan for national retain volume (units) and price of avocados 

In [3]:
avocado_df = pd.read_csv('avocado-updated-2020.csv')

In [95]:
avocado_df.head()

,date,average_price,total_volume,4046,4225,4770,total_bags,small_bags,large_bags,xlarge_bags,type,year,geography,total_revenue
0,2015-01-04,1.22,40873.28,2819.50,28287.42,49.90,9716.46,9186.93,529.53,0.0,conventional,2015,Albany,49865.4016
1,2015-01-04,1.79,1373.95,57.42,153.88,0.00,1162.65,1162.65,0.00,0.0,organic,2015,Albany,2459.3705
2,2015-01-04,1.00,435021.49,364302.39,23821.16,82.15,46815.79,16707.15,30108.64,0.0,conventional,2015,Atlanta,435021.4900
3,2015-01-04,1.76,3846.69,1500.15,938.35,0.00,1408.19,1071.35,336.84,0.0,organic,2015,Atlanta,6770.1744
4,2015-01-04,1.08,788025.06,53987.31,552906.04,39995.03,141136.68,137146.07,3990.61,0.0,conventional,2015,Baltimore/Washington,851067.0648


In [5]:
avocado_df.describe()

,average_price,total_volume,4046,4225,4770,total_bags,small_bags,large_bags,xlarge_bags,year
count,33045.000000,3.304500e+04,3.304500e+04,3.304500e+04,3.304500e+04,3.304500e+04,3.304500e+04,3.304500e+04,3.304500e+04,33045.000000
mean,1.379941,9.683997e+05,3.023914e+05,2.797693e+05,2.148255e+04,3.646735e+05,2.501980e+05,1.067329e+05,7.742585e+03,2017.460887
std,0.378972,3.934533e+06,1.301026e+06,1.151052e+06,1.001607e+05,1.564004e+06,1.037734e+06,5.167226e+05,4.819803e+04,1.699747
min,0.440000,8.456000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2015.000000
25%,1.100000,1.511895e+04,7.673100e+02,2.712470e+03,0.000000e+00,9.121860e+03,6.478630e+03,4.662900e+02,0.000000e+00,2016.000000
50%,1.350000,1.291170e+05,1.099477e+04,2.343600e+04,1.780900e+02,5.322224e+04,3.687699e+04,6.375860e+03,0.000000e+00,2017.000000
75%,1.620000,5.058285e+05,1.190219e+05,1.352389e+05,5.096530e+03,1.744314e+05,1.206624e+05,4.041723e+04,8.044400e+02,2019.000000
max,3.250000,6.371614e+07,2.274362e+07,2.047057e+07,2.546439e+06,3.168919e+07,2.055041e+07,1.332760e+07,1.403184e+06,2020.000000


## 2. EDA - check for missing values and clean 

In [6]:
avocado_df.isnull().sum()

date             0
average_price    0
total_volume     0
4046             0
4225             0
4770             0
total_bags       0
small_bags       0
large_bags       0
xlarge_bags      0
type             0
year             0
geography        0
dtype: int64

### Inspect the geography column, as this is the one we are interested in for the task of deploying headcount across the US

In [7]:
avocado_df['geography'].unique()

array(['Albany', 'Atlanta', 'Baltimore/Washington', 'Boise', 'Boston',
       'Buffalo/Rochester', 'California', 'Charlotte', 'Chicago',
       'Cincinnati/Dayton', 'Columbus', 'Dallas/Ft. Worth', 'Denver',
       'Detroit', 'Grand Rapids', 'Great Lakes', 'Harrisburg/Scranton',
       'Hartford/Springfield', 'Houston', 'Indianapolis', 'Jacksonville',
       'Las Vegas', 'Los Angeles', 'Louisville', 'Miami/Ft. Lauderdale',
       'Midsouth', 'Nashville', 'New Orleans/Mobile', 'New York',
       'Northeast', 'Northern New England', 'Orlando', 'Philadelphia',
       'Phoenix/Tucson', 'Pittsburgh', 'Plains', 'Portland',
       'Raleigh/Greensboro', 'Richmond/Norfolk', 'Roanoke', 'Sacramento',
       'San Diego', 'San Francisco', 'Seattle', 'South Carolina',
       'South Central', 'Southeast', 'Spokane', 'St. Louis', 'Syracuse',
       'Tampa', 'Total U.S.', 'West', 'West Tex/New Mexico'], dtype=object)

In [8]:
# Drop generic / non descriptive locations - these are not useful for our task 

avocado_df = avocado_df.loc[~avocado_df['geography'].isin(['Total U.S.', 'Northeast', 'Midsouth', 'South Central', 'Southeast', 'West', 'California', 
                                                           'Northern New England', 'South Carolina'])]

## 3. Feature Engineering - total revenue 

In [9]:
# Locations with the highest total revenue will those where Olaplex should deploy headcount

avocado_df['total_revenue'] = avocado_df['total_volume'] * avocado_df['average_price']

In [10]:
# Group by geography to calculate total revenue, total volume, and average price (across both avocado types)

top_20_summary = avocado_df.groupby('geography').agg(
    total_volume=('total_volume', 'sum'),
    avg_price=('average_price', 'mean'),
    total_revenue=('total_revenue', 'sum')
).reset_index()

# Sort by total revenue in descending order and extract the top 20 locations
top_20_summary = top_20_summary.sort_values(by='total_revenue', ascending=False).head(20).reset_index(drop=True)

In [11]:
top_20_summary

,geography,total_volume,avg_price,total_revenue
0,Great Lakes,1.220448e+09,1.310572,1.387575e+09
1,Los Angeles,9.593505e+08,1.311013,9.985289e+08
2,New York,5.282452e+08,1.659477,7.218311e+08
3,Plains,6.352576e+08,1.351324,7.108276e+08
4,San Francisco,2.742169e+08,1.759967,3.794715e+08
5,Baltimore/Washington,2.767646e+08,1.474820,3.654018e+08
6,Dallas/Ft. Worth,4.362042e+08,1.076160,3.603434e+08
7,Chicago,2.576650e+08,1.524265,3.383896e+08
8,Houston,4.208438e+08,1.075752,3.367408e+08
9,Denver,2.779295e+08,1.263088,2.999316e+08


### As another option, if Olaplex wanted to balance between demand and revenue, we could create a weighted metric based on if Olaplex values total volume more than total revnue, or vice versa 

In [96]:
# # Example weights for the composite score 
total_volume_weight = 0.7  # Weight for total volume (emphasis on demand)
revenue_weight = 0.3  # Weight for total revenue (profitability)

# Calculate the composite score
top_20_summary_with_composite = top_20_summary.copy()
top_20_summary_with_composite['composite_score'] = (
     total_volume_weight * (top_20_summary_with_composite['total_volume'] / top_20_summary_with_composite['total_volume'].max()) + 
     revenue_weight * (top_20_summary_with_composite['total_revenue'] / top_20_summary_with_composite['total_revenue'].max())
 )

# Sort by the composite score in descending order
top_20_summary_with_composite = top_20_summary_with_composite.sort_values(by='composite_score', ascending=False)

# Since we don't have that information for this task, stick with using total revenue as an indication of where to place headcount

## 4. Visualize results 

## 4a. Top 20 locations with the highest historical total revenue - these are the locations Olaplex should deploy the sales headcount 

In [106]:
fig = px.bar(
    top_20_summary,
    x="total_revenue",
    y="geography",  
    orientation="h",
    title="Top 20 Regions by Total Avocado Revenue from 2015 - 2020",
    labels={"total_revenue": "Total Revenue (in USD)", "geography": "Region"},
    text="total_revenue"
)

fig.update_layout(
    xaxis=dict(title="Total Revenue (in USD)", tickformat="$.2s"),
    yaxis=dict(title="Region", categoryorder="total ascending"),  # Sort by revenue
    template="plotly_white",
    height=640  
)

# Add hover for exact values
fig.update_traces(
    texttemplate="%{text:.2s}",  
    hovertemplate=(
        "<b>%{y}</b><br>"
        "Total Revenue: $%{x:,.2f}<br>"
    )
)

fig.show()

In [14]:
# Create a second dataframe with all the original columns for expanded (and any other) visualizations

top_20_detailed = avocado_df[avocado_df['geography'].isin(top_20_summary['geography'])]

In [15]:
# Save the outputs
# top_20_summary.to_csv("/Users/hannahwhitmore/desktop/top_20_summary.csv", index=False)
# top_20_detailed.to_csv("/Users/hannahwhitmore/desktop/top_20_detailed.csv", index=False)

In [16]:
# Helper function for formatting

def format_rev(value):
    if value >= 1e9:
        return f"{value / 1e9:.1f}B"  
    elif value >= 1e6:
        return f"{value / 1e6:.1f}M"  
    elif value >= 1e3:
        return f"{value / 1e3:.1f}K" 
    else:
        return f"{value:.1f}"      

In [17]:
# Need to map cities to their state abbrevations for Plotly's geographic library

region_to_state = {
    'Atlanta': 'GA',
    'Baltimore/Washington': 'MD',
    'Boston': 'MA',
    'Chicago': 'IL',
    'Dallas/Ft. Worth': 'TX',
    'Denver': 'CO',
    'Great Lakes': 'MI',  
    'Houston': 'TX',
    'Los Angeles': 'CA',
    'Miami/Ft. Lauderdale': 'FL',
    'New York': 'NY',
    'Philadelphia': 'PA',
    'Phoenix/Tucson': 'AZ',
    'Plains': 'KS',  
    'Portland': 'OR',
    'Sacramento': 'CA',
    'San Diego': 'CA',
    'San Francisco': 'CA',
    'Seattle': 'WA',
    'West Tex/New Mexico': 'TX',  # Or NM
}

## 4b. Top 20 total revenue on the map of the US 

In [18]:
map_data = top_20_detailed.groupby('geography').agg(
    total_revenue=('total_revenue', 'sum'),
    total_volume=('total_volume', 'sum')
).reset_index()

map_data['state'] = map_data['geography'].map(region_to_state)
map_data['Total Revenue'] = map_data['total_revenue'].apply(format_rev)
map_data['Total Volume'] = map_data['total_volume'].apply(format_rev)

aggregated_map_fig = px.scatter_geo(
    map_data,
    locations="state",
    locationmode="USA-states",
    size="total_revenue",
    color="total_revenue",  
    hover_name="geography",
    hover_data={
        "Total Revenue": True, 
        "Total Volume": True,
        "total_revenue": False,
        "total_volume": False,
    },
    title="Revenue Distribution Across Top 20 Regions (2015 - 2020)",
    projection="albers usa",
)

aggregated_map_fig.show()

## 4c. Top 20 total revenues by avocado type

In [88]:
type_fig = px.bar(
    top_20_detailed.groupby(['geography', 'type'])['total_revenue']
    .sum()
    .reset_index()
    .sort_values(by="total_revenue", ascending=False),
    x="total_revenue",
    y="geography",
    color="type",
    orientation="h",
    title="Top 20 Regions Total Revnue (2015 - 2020) By Avocado Type",
    labels={"total_revenue": "Total Revenue (USD)", "geography": "Region"},
    height=600
)
type_fig.show()

## 4d. Top 20 revenues by year

In [89]:
line_data = top_20_detailed.groupby(['year', 'geography'])['total_revenue'].sum().reset_index()
line_data['Total Revenue'] = line_data['total_revenue'].apply(format_rev)

In [91]:
line_data.to_csv('/Users/hannahwhitmore/desktop/linedata.csv')

In [108]:
line_data = top_20_detailed.groupby(['year', 'geography'])['total_revenue'].sum().reset_index()
line_data['Total Revenue'] = line_data['total_revenue'].apply(format_rev)

years = sorted(line_data['year'].unique())

line_fig = px.line(
    line_data,
    x="year",
    y="total_revenue",
    color="geography",
    line_group="geography",
    title="Yearly Revenue Trends by Geography",
    labels={"total_revenue": "Total Revenue (USD)", "year": "Year"},
    hover_data={"Total Revenue": True, "total_revenue": False}  
)

line_fig.update_layout(
    xaxis=dict(
        tickmode="array",
        tickvals=years,  
        ticktext=[str(year) for year in years],  
    ),
    height=600
)

line_fig.show()

## 4e. Top Revenues Across Years - Dynamic 

In [49]:
bar_data = top_20_detailed.groupby(['geography', 'year']).agg(
    total_revenue=('total_revenue', 'sum')
).reset_index()
bar_data['total_revenue_formatted'] = bar_data['total_revenue'].apply(format_rev)

bar_fig = px.bar(
    bar_data,
    x="total_revenue",
    y="geography",
    color="geography",
    animation_frame="year",
    orientation="h",
    title="Top Regions by Revenue per Year",
    labels={"total_revenue": "Total Revenue (USD)", "geography": "Region"},
    text="total_revenue_formatted",
    hover_data={"total_revenue_formatted": False}
)

max_revenue = bar_data['total_revenue'].max()
bar_fig.update_layout(
    xaxis=dict(
        title="Total Revenue (USD)",
        tickformat="$.2s",
        range=[0, max_revenue * 1.1]  # Extend slightly beyond max value for padding
    ),
    yaxis=dict(title="Region", categoryorder="total ascending"),
    template="plotly_white",
    height=700,
    showlegend=False,
    updatemenus=[{
        "buttons": [
            {
                "args": [None, {"frame": {"duration": 1500, "redraw": True}, "fromcurrent": True}],
                "label": "Play",
                "method": "animate",
            },
            {
                "args": [[None], {"frame": {"duration": 0, "redraw": True}, "mode": "immediate", "transition": {"duration": 0}}],
                "label": "Pause",
                "method": "animate",
            },
        ],
        "direction": "left",
        "pad": {"r": 10, "t": 87},
        "showactive": False,
        "type": "buttons",
        "x": 0.1,
        "xanchor": "right",
        "y": 0,
        "yanchor": "top",
    }]
)
bar_fig.show()

## 4f. Price vs Volume Across Top 20 Regions

In [94]:
scatter_data = top_20_detailed.groupby(['geography']).agg(
    avg_price=('average_price', 'mean'),
    total_volume=('total_volume', 'sum')
).reset_index()

scatter_data['Average Price (USD)'] = scatter_data['avg_price'].round(2)  
scatter_data['Total Volume'] = scatter_data['total_volume'].apply(format_rev)


scatter_fig = px.scatter(
    scatter_data,
    x="avg_price",
    y="total_volume",
    size="total_volume",
    color="geography",
    hover_name="geography",
    hover_data={
        "Total Volume": True,
        "geography": False,
        "avg_price": False,
        "Average Price (USD)": True,
        "total_volume": False,
    },
    title="Average Price vs Total Volume",
    height=600
)

scatter_fig.show()

In [50]:
# Regions with high volume despite higher prices may represent premium markets

### 4g. Average price over time by avocado type

In [73]:
line_price_data = top_20_detailed.groupby(['year', 'type']).agg(
    avg_price=('average_price', 'mean')
).reset_index()
line_price_data['avg_price'] = line_price_data['avg_price'].round(2)  


years = sorted(line_price_data['year'].unique())

line_price_fig = px.line(
    line_price_data,
    x="year",
    y="avg_price",
    color="type",
    title="Trends in Average Price Over Time by Type",
    labels={"avg_price": "Average Price (USD)", "year": "Year"}
)

line_price_fig.update_layout(
    xaxis=dict(
        tickmode="array",
        tickvals=years,  
        ticktext=[str(year) for year in years],  
    ),
    height=550
)

line_price_fig.show()

In [74]:
# Yearly trends in average_price for conventional and organic avocados
# Insights into whether organic avocados command a consistent premium

In [50]:
plu_data = top_20_detailed[['4046', '4225', '4770', 'geography']].groupby('geography').sum().reset_index()

plu_data = pd.melt(plu_data, id_vars=['geography'], var_name='PLU', value_name='volume')

treemap_fig = px.treemap(
    plu_data,
    path=["geography", "PLU"],
    values="volume",
    title="PLU Code Breakdown by Region",
    labels={"volume": "Total Volume"}
)

treemap_fig.show()

In [ ]:
# How different PLU codes contribute to total volume within each region.
# Regions with diverse vs. concentrated PLU preferences.